ImportError: The descartes package is required for plotting polygons in geopandas<br>
-> conda install descartes

In [1]:
import geopandas as gpd
import pandas as pd

# Read shapefile
gdf = gpd.read_file("./shapefile/city2021_ok.shp", encoding='utf-8')

# shapefileのgeometryを単純化（simplify）する。
cities = gdf.geometry
simplified = cities.simplify(tolerance=0.00085, preserve_topology=True)

# geometryデータを書き換える。
gdf["geometry"] = simplified

# Read csv data
df = pd.read_csv("./csv/国勢調査2020_夜間人口.csv").astype(str)

# 都市コードをゼロ埋めの5桁表示にする。
code = []
for i in df["市区町村コード"]:
    code.append(i.zfill(5))
df.loc[:,"NEW_ID"] = code

# データフレームを結合する。
merged_data = pd.merge(gdf,df, left_on='NEW_ID', right_on='NEW_ID',how="outer")
gdf2 = gpd.GeoDataFrame(merged_data)
print(gdf2)

# geometryが空白の行を削除する。
gdf3 = gdf2.dropna(subset=['geometry'])

# 面積を計測する。
areas = []
for name, geo in zip(gdf3["name1"], gdf3["geometry"]):
    # 面積出力
    area = "{:.2f}".format(float(geo.area*10000))
    areas.append(area)

# 面積の列を追加する。
gdf3.loc[:,"面積"] = areas

# 人口密度を算出する。
densities = []
for i, j in zip(gdf3["夜間人口"] , gdf3["面積"]):
    try:
        densities.append(float(i)/float(j))
    except:
        densities.append(0)

# 人口密度の列を追加する。
gdf3.loc[:,"人口密度"] = densities

# Export as geojson
gdf3.to_file("population_density_japan.geojson", driver='GeoJSON')

print("completed")


     NEW_ID  ken   gun name1 name2     区分   地域  都道府  都市名   タイプ     市区町  \
0     01100  北海道  None   札幌市   中央区  138.0  北海道  北海道   札幌  政令指定  011002   
1     01202  北海道  None   函館市  None  138.0  北海道  北海道   函館   その他  012025   
2     01204  北海道  None   旭川市  None  138.0  北海道  北海道   旭川   その他  012041   
3     01206  北海道  None   釧路市  None  138.0  北海道  北海道   釧路   その他  012068   
4     01213  北海道  None  苫小牧市  None  138.0  北海道  北海道  苫小牧   その他  012131   
...     ...  ...   ...   ...   ...    ...  ...  ...  ...   ...     ...   
1974  43105  NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN   NaN     NaN   
1975  44000  NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN   NaN     NaN   
1976  45000  NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN   NaN     NaN   
1977  46000  NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN   NaN     NaN   
1978  47000  NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN   NaN     NaN   

                                               geometry     年次 市区町村コード  \
0     POLYGON ((141.40864 43.18412, 1

c:\Users\nyama\anaconda\envs\MyPlatform\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\nyama\anaconda\envs\MyPlatform\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\nyama\anaconda\envs\MyPlatform\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will 

completed
